In [1]:
from faker import Faker
import sqlite3
import pandas as pd
import random
import datetime

In [ ]:
fake = Faker()
NUM_AUTHORS = 4

In [ ]:
# Create a new database
conn = sqlite3.connect('fake_data.db')
c = conn.cursor()

In [ ]:
# Create the blog post table
c.execute('''CREATE TABLE posts(
                id INTEGER PRIMARY KEY,          
                author_id INTEGER,
                published text, 
                title text, 
                body text)
          ''')

# Create the author table
c.execute('''CREATE TABLE authors(
                id INTEGER PRIMARY KEY, 
                name text, 
                email text)
          ''')

In [ ]:
# Create our fake authors
authors = []
for i in range(NUM_AUTHORS):
    authors.append(fake.name())

In [ ]:
# Write the authors to the database
for i, author in enumerate(authors):
    c.execute('''INSERT INTO authors (id, name, email) 
                  VALUES (?, ?, ?)''', (i, author, fake.email()))

conn.commit()

In [ ]:
# Create the fake posts
# Give each author between 5 and 10 posts
# Randomize the date to be within the last year
for i, author in enumerate(authors):
    num_posts = random.randint(5, 10)

    for j in range(num_posts):
        published = fake.date_time_this_year()
        title = fake.sentence()
        body = fake.text()
        
        c.execute('''INSERT INTO posts (author_id, published, title, body) 
                      VALUES (?, ?, ?, ?)''', (i, published, title, body))
        
conn.commit()

In [ ]:
# Load up the table in pandas to check it out
df = pd.read_sql_query("SELECT * FROM posts", conn)
df.head()

In [ ]:
# Close the connection
conn.close()